<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Bayesian Personalized Ranking (BPR)

This notebook serves as an introduction to Bayesian Personalized Ranking (BPR) model for implicit feedback.  In this tutorial, we focus on learning the BPR model using matrix factorization approach, hence, the model is sometimes also named BPRMF.

The implementation of the model is from [Cornac](https://github.com/PreferredAI/cornac), which is a framework for recommender systems with a focus on models leveraging auxiliary data (e.g., item descriptive text and image, social network, etc).

## 0 Global Settings and Imports

## 1 BPR Algorithm

### 1.1 Personalized Ranking from Implicit Feedback

The task of personalized ranking aims at providing each user a ranked list of items (recommendations).  This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks).  The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$.  BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of *positive* and *negative* items.  The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$

where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a *positive item* and $j$ is a *negative item*).


### 1.2 Objective Function

From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i >_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid >_{u}) \propto p(i >_{u} j \mid \Theta) \times p(\Theta)$$

The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(>_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i >_{u} j \mid \Theta) $$

The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i >_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$

where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$

The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$.  Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$

To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$.  Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$

where $\lambda_\Theta$ are the model specific regularization paramerters.


### 1.3 Learning with Matrix Factorization

#### Stochastic Gradient Descent

As the defined objective function is differentible, gradient descent based method for optimization is naturally adopted.  The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} & = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
& \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$

Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model.  For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$

#### Matrix Factorization for Preference Approximation

As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function.  First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$

The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective.  The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$

The  derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  & \text{if } \theta = w_{uf} \\
    w_{uf}             & \text{if } \theta = h_{if} \\
    -w_{uf}            & \text{if } \theta = h_{jf} \\
    0                  & \text{else}
\end{cases}
$$

In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $.  For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.


In [72]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '20m'

# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 20000
NUM_EPOCHS = 20

In [73]:
import sys
sys.path.append("../../")
import os
import cornac
import papermill as pm
import pandas as pd
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.cornac.cornac_utils import predict_ranking
from reco_utils.common.timer import Timer
from reco_utils.common.constants import SEED

print("System version: {}".format(sys.version))

System version: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [74]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset.download_utils import maybe_download
from reco_utils.dataset.python_splitters import (
    python_random_split, 
    python_chrono_split, 
    python_stratified_split
)
from reco_utils.dataset.spark_splitters import (
    spark_random_split, 
    spark_chrono_split, 
    spark_stratified_split,
    spark_timestamp_split
)

print("System version: {}".format(sys.version))


System version: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## 2 Cornac implementation of BPR

BPR is implemented in the [Cornac](https://cornac.readthedocs.io/en/latest/index.html) framework as part of the model collections.
* Detailed documentations of the BPR model in Cornac can be found [here](https://cornac.readthedocs.io/en/latest/models.html#bayesian-personalized-ranking-bpr).
* Source codes of the BPR implementation is available on the Cornac Github repository, which can be found [here](https://github.com/PreferredAI/cornac/blob/master/cornac/models/bpr/recom_bpr.pyx).


## 3 Cornac BPR movie recommender


### 3.1 Load and split data
为了评估项目推荐的表现情况，我们采取随机分割的python工具`python_random_split` 来保持一致性。数据被随机地分割成训练和测试的两个集合，比例为75/25.
Cornac还提供了许多其他的用以评估的工具方案。[built-in schemes](https://cornac.readthedocs.io/en/latest/eval_methods.html) 

将数据导入
预览前几行的数据信息。
然后对数据进行分割。

In [1]:
import pandas as pd
#获取课程内容
filepath_section = "/Users/apple/Recommenders/examples/02_model_collaborative_filtering/moodle/section课程内容与时间.csv"
data_sectionTime = pd.read_csv(filepath_section,encoding='utf-8')
data_sectionTime.head()


,beidawlf_01_01,1.1,00:00:00.000,1900-01-02 04:20:00.000,软件及软件工程的相关概念(1),beidawlf_01_01.1
0,beidawlf_01_01_01,1.1,00:00:00.000,01:29:00.000,第一讲主要内容概述,beidawlf_01_01_01
1,beidawlf_01_01_02,1.2,01:29:00.000,11:50:00.000,软件的概念,beidawlf_01_01_02
2,beidawlf_01_01_03,1.2,11:50:00.000,17:30:00.000,软件的分类,beidawlf_01_01_03
3,beidawlf_01_01_04,1.2,17:30:00.000,21:40:00.000,软件研究的三个层次,beidawlf_01_01_04
4,beidawlf_01_01_05,1.2,21:40:00.000,1900-01-01 00:18:00.000,软件发展的阶段,beidawlf_01_01_05


In [2]:
#获得学生行为数据
filepath_student ='/Users/apple/Recommenders/examples/02_model_collaborative_filtering/moodle/学生行为记录2.csv'
data_student = pd.read_csv(filepath_student,encoding='gbk')
data_student.head()

,id,学号,课程id,行为编号,开始时间,持续时间,行为发生时间
0,1,C9C2FE704F86435C,zhedacy_01_02_02,1,\N,\N,2017/2/23 2:01
1,2,FA7F9E298B05D28A,beidawlf_01_02_02,1,\N,\N,2017/2/23 8:33
2,3,FA7F9E298B05D28A,beidawlf_01_02_02,2,00:00.0,02:00.0,2017/2/23 8:33
3,4,FA7F9E298B05D28A,beidawlf_01_02_02,3,02:00.0,08:00.0,2017/2/23 8:33
4,5,FA7F9E298B05D28A,beidawlf_01_02_02,2,02:00.0,48:00.0,2017/2/23 8:44


In [3]:
data_student.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679078 entries, 0 to 679077
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      679078 non-null  int64 
 1   学号      679078 non-null  object
 2   课程id    679078 non-null  object
 3   行为编号    679078 non-null  int64 
 4   开始时间    679078 non-null  object
 5   持续时间    679078 non-null  object
 6   行为发生时间  679078 non-null  object
dtypes: int64(2), object(5)
memory usage: 36.3+ MB


In [68]:
DATA_PATH = "/Users/apple/Recommenders/examples/02_model_collaborative_filtering/moodle"

COL_USER = "学号"
COL_ITEM = "课程id"
COL_RATING = "持续时间"
COL_PREDICTION = "Rating"
COL_TIMESTAMP = "行为发生时间"
print(
    #"Total number of ratings are\t{}".format(data_student[COL_RATING]),
    "Total number of users are\t{}".format(data_student[COL_USER].nunique()),
    "Total number of items are\t{}".format(data_student[COL_ITEM].nunique()),
    sep="\n"
)

Total number of users are	331
Total number of items are	1421


In [ ]:
#获得课程项目的名称地图
data_item=data_sectionTime[['beidawlf_01_01','软件及软件工程的相关概念(1)']]
data_item.head()
#获得课程项目的视频地址
data_video=data_sectionTime[['beidawlf_01_01']]
data_video.head()
data_sectionTime.head()

data_studentMap = data_student[['学号','学号']]


In [44]:
data = data_student[(data_student["行为编号"] == 7)]
data[COL_TIMESTAMP] = pd.to_datetime(data[COL_TIMESTAMP],format = '%Y/%m/%d %H:%M')
data[COL_TIMESTAMP].dtypes
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41154 entries, 10 to 679077
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      41154 non-null  int64         
 1   学号      41154 non-null  object        
 2   课程id    41154 non-null  object        
 3   行为编号    41154 non-null  int64         
 4   开始时间    41154 non-null  object        
 5   持续时间    41154 non-null  object        
 6   行为发生时间  41154 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 2.5+ MB


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#清洗文本
i =0
for each in data["持续时间"].values:
    #each.replace("-",'')
    if(type(each) != int):
        ea = int(each.split(":")[0])
        data[COL_RATING].values[i] =ea
    i = i+1



In [63]:
#清洗非正常格式的数据
i=0
for each in data["持续时间"].values:
    if(type(each) != int):
        data[COL_RATING].values[i] = 0;

    i = i+1


In [65]:
data.info()
data["持续时间"].values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41154 entries, 10 to 679077
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      41154 non-null  int64         
 1   学号      41154 non-null  object        
 2   课程id    41154 non-null  object        
 3   行为编号    41154 non-null  int64         
 4   开始时间    41154 non-null  object        
 5   持续时间    41154 non-null  object        
 6   行为发生时间  41154 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 2.5+ MB


array([8, 1, 1, ..., 0, 0, 0], dtype=object)

In [80]:
#如何选择rating的数据？
#获得需要的三元组

data_uir = data[['学号','课程id','持续时间']] #,"行为发生时间"
data_uir.head()

,学号,课程id,持续时间
10,929C2CD2FD0EF9A3,beidawlf_01_02_05,8
19,533FA407DBA0C51C,beidawlf_05_03_09,1
21,533FA407DBA0C51C,beidawlf_04_01_08,1
23,533FA407DBA0C51C,beidawlf_04_01_09,0
25,533FA407DBA0C51C,beidawlf_04_01_10,0


In [70]:
#reverse_data_uir = pd.DataFrame(data_uir.values.T, index=data_uir.columns, columns=data_uir.index)

#reverse_data_uir.head()

In [99]:
train, test = python_random_split(data_uir, 0.75)
#对训练集进行转置处理
#train2 = pd.DataFrame(train.values.T, index=train.columns, columns=train.index)
#train2.head()

### 3.2 Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.
为了能够成功使用Cornac来建立模型，需要将数据集建立成一个对象。
数据转化，数据集还提供了一系列工具来在数据之中浏览循环，以及许多用来消极取样的工具。 negative sampling techniques.

创建train_set对象， ##中间的参数没有看懂
显示两个主要集合的数量

In [84]:
train_set = cornac.data.trainset.MatrixTrainSet.from_uir(train.itertuples(index=False) )

print('Number of users: {}'.format(train_set.num_users),)
print('Number of items: {}'.format(train_set.num_items))
#.itertuples(index=False)?what's

Number of users: 330
Number of items: 803


### 3.3 Train the BPR model

BPR具有一些我们需要考虑的重要参数：

-`k`：控制潜在空间的尺寸（即向量$ w_u $和$ h_i $的大小）。
-`max_iter`：定义SGD过程的迭代次数。
-“ learning_rate”：控制渐变更新规则中的步长$ \ alpha $。
-`lambda_reg`：控制目标函数中的L2-Regularization $ \ lambda $。

注意，`k`和`max_iter`的不同值会影响训练时间。

我们在这里将k设置为200，max_iter设置为100，learning_rate设置为0.01，lambda_reg设置为0.001。要训​​练模型，我们只需要调用`fit（）`方法。

In [85]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [86]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 20/20 [00:05<00:00,  3.39it/s, correct=90.20%, skipped=98.63%] 

Optimization finished!
Took 6.4717 seconds for training.


### 3.4 Prediction and Evaluation

现在我们的模型已经训练完毕，我们可以生成排名列表以供推荐。 Cornac中的每个推荐器模型都提供“ rate（）”和“ rank（）”方法来预测给定用户的项目额定值以及项目排名列表。为了利用当前的评估方案，我们将通过cornac_utils中的`predict（）`和`predict_ranking（）`函数来生成预测。

请注意，BPR模型是为项目排名有效设计的。因此，我们仅使用排名指标来衡量效果。模型得以训练成功之后，可以生成推荐的排序列表。利用test集合，选取其中所有的预测结果，取名为'prediction'列，选取其中k值。

评估其中的Map,Ndcg,Precision, Recall@k，将评估结果存储在pm之中

In [93]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='学号', itemcol='课程id', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 1.2539 seconds for prediction.


In [95]:
userID ='学号'
itemID ='课程id'
rating = '课程持续时间'

In [102]:
all_predictions.head()

,学号,课程id,prediction
30865,4F81E6DA41395DA8,beidawlf_03_01_01,0.082525
30866,4F81E6DA41395DA8,beidawlf_08_05,0.666111
30867,4F81E6DA41395DA8,beidawlf_01_02_01,-0.266256
30868,4F81E6DA41395DA8,beidawlf_05_03,3.265529
30869,4F81E6DA41395DA8,beidawlf_05_01_11,1.637036


In [ ]:
#存储到仓库中
all_predictions.to_csv('all_predictions.csv')

In [97]:
k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Missing column: userID in DataFrame
Missing column: itemID in DataFrame
Missing column: rating in DataFrame


ValueError: Missing columns in true rating DataFrame

In [ ]:
# Record results with papermill for tests
pm.record("map", eval_map)
pm.record("ndcg", eval_ndcg)
pm.record("precision", eval_precision)
pm.record("recall", eval_recall)

## References

1. Rendle, S., Freudenthaler, C., Gantner, Z., & Schmidt-Thieme, L. (2009, June). BPR: Bayesian personalized ranking from implicit feedback. https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf
2. Pan, R., Zhou, Y., Cao, B., Liu, N. N., Lukose, R., Scholz, M., & Yang, Q. (2008, December). One-class collaborative filtering. https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/04781145.pdf
3. **Cornac** - A Comparative Framework for Multimodal Recommender Systems. https://cornac.preferred.ai/